# Reconocimiento de Entidades Nombradas
## Ejercicio Práctico - Procesamiento de Lenguaje Natural

**Objetivos de Aprendizaje:**
1. Implementar NER usando modelos pre-entrenados en español
2. Crear interfaces interactivas con Gradio
3. Comparar enfoques: Transformers vs API Gemini
4. Desarrollar prototipos rápidos para aplicaciones de PLN

---
**Entorno recomendado:** Google Colab o Amazon SageMaker Studio

**Tiempo estimado:** 60-90 minutos

## Instalación de Dependencias

In [24]:
# Instalación de librerías necesarias
%%capture
!pip install -q transformers torch gradio google-genai

In [25]:
# Verificar instalación
import sys
print(f"Python: {sys.version}")
print("Todas las dependencias instaladas correctamente")

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Todas las dependencias instaladas correctamente


## Configuración de APIs

### Para Google Colab:
1. Andá a la barra lateral izquierda y hacé clic en 🔑 (Secrets)
2. Agrega una nueva clave: `GOOGLE_API_KEY`
3. Pega tu API key de Google AI Studio

### Para SageMaker Studio:
1. Configura las variables de entorno en tu instancia
2. O usa el método de input manual más abajo

In [26]:
import os
import warnings
warnings.filterwarnings('ignore')

# Configuración de API Key para Gemini
try:
    # Método 1: Google Colab Secrets
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    print("API Key cargada desde Google Colab Secrets")
except:
    # Método 2: Variables de entorno (SageMaker)
    GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')
    if GOOGLE_API_KEY:
        print("API Key cargada desde variables de entorno")
    else:
        print("No se encontró GOOGLE_API_KEY")
        print("Podes continuar solo con la parte de Transformers")

API Key cargada desde Google Colab Secrets


---
# PARTE 1: NER con Transformers de Hugging Face

Utilizaremos un modelo especializado en español que puede identificar:
- **PER**: Personas
- **LOC**: Lugares
- **ORG**: Organizaciones  
- **MISC**: Misceláneo

In [27]:
from transformers import pipeline
import torch

# Verificar disponibilidad de GPU
device = 0 if torch.cuda.is_available() else -1
print(f"🖥️  Dispositivo: {'GPU' if device == 0 else 'CPU'}")

# Cargar modelo de NER en español
print("📥 Cargando modelo de NER para español...")
MODEL_NAME = "mrm8488/bert-spanish-cased-finetuned-ner"

try:
    ner_pipeline = pipeline(
        "ner",
        model=MODEL_NAME,
        aggregation_strategy="simple",
        device=device
    )
    print(f"Modelo {MODEL_NAME} cargado exitosamente")
except Exception as e:
    print(f"❌ Error al cargar modelo: {e}")
    ner_pipeline = None

🖥️  Dispositivo: GPU
📥 Cargando modelo de NER para español...


Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


Modelo mrm8488/bert-spanish-cased-finetuned-ner cargado exitosamente


In [28]:
# Texto de ejemplo con contexto argentino
texto_ejemplo = """
Hola, soy María González y trabajo en la Universidad de Buenos Aires.
Vivo en el barrio de San Telmo y mi empresa favorita es MercadoLibre.
La semana pasada visité el Obelisco con mi amigo Carlos Pérez,
quien trabaja en Google Argentina. Nos encontramos en la estación
Constitución del subte y fuimos a comer un asado en La Boca.
"""

In [29]:
def analizar_entidades_transformers(texto):
    """Procesa texto y extrae entidades usando Transformers"""
    if not ner_pipeline:
        return []

    entidades = ner_pipeline(texto)

    # Formatear resultados
    resultados = []
    for ent in entidades:
        resultados.append({
            'texto': ent['word'],
            'etiqueta': ent['entity_group'],
            'confianza': round(ent['score'], 3),
            'posicion': (ent['start'], ent['end'])
        })

    return resultados

In [30]:
# Probar el modelo
print("🔍 Analizando texto de ejemplo...")
print(f"📝 Texto: {texto_ejemplo.strip()}")
print("\n📊 Entidades encontradas:")

entidades_encontradas = analizar_entidades_transformers(texto_ejemplo)
for ent in entidades_encontradas:
    print(f"  • {ent['texto']} → {ent['etiqueta']} (confianza: {ent['confianza']})")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


🔍 Analizando texto de ejemplo...
📝 Texto: Hola, soy María González y trabajo en la Universidad de Buenos Aires.
Vivo en el barrio de San Telmo y mi empresa favorita es MercadoLibre.
La semana pasada visité el Obelisco con mi amigo Carlos Pérez,
quien trabaja en Google Argentina. Nos encontramos en la estación
Constitución del subte y fuimos a comer un asado en La Boca.

📊 Entidades encontradas:
  • María González → PER (confianza: 0.9990000128746033)
  • Universidad de Buenos Aires → ORG (confianza: 0.9990000128746033)
  • San Telmo → LOC (confianza: 0.9980000257492065)
  • MercadoLibre → ORG (confianza: 0.996999979019165)
  • Obelisco → LOC (confianza: 0.9959999918937683)
  • Carlos Pérez → PER (confianza: 1.0)
  • Google Argentina → ORG (confianza: 0.9860000014305115)
  • Constitución → LOC (confianza: 0.9950000047683716)
  • La Boca → LOC (confianza: 1.0)


---
# PARTE 2: NER con API de Gemini

Utilizaremos la API de Gemini para un análisis más detallado y contextual.

In [31]:
# Configurar cliente Gemini
cliente_gemini = None

if GOOGLE_API_KEY:
    try:
        from google import genai
        cliente_gemini = genai.Client(api_key=GOOGLE_API_KEY)
        print("Cliente Gemini configurado correctamente")
    except Exception as e:
        print(f"Error al configurar Gemini: {e}")
else:
    print("API Key de Gemini no disponible")
    print("Podes obtener una gratis en: https://ai.google.dev/")

Cliente Gemini configurado correctamente


In [32]:
def analizar_entidades_gemini(texto):
    """Analiza entidades usando Gemini API"""
    if not cliente_gemini:
        return "❌ Cliente Gemini no disponible"

    prompt = f"""
    Extraé todas las entidades nombradas del siguiente texto en español argentino y clasificálas:

    CATEGORÍAS:
    - PERSONA: Nombres de personas
    - LUGAR: Ciudades, países, barrios, direcciones, lugares específicos
    - ORGANIZACIÓN: Empresas, universidades, instituciones
    - MISCELÁNEO: Otros nombres propios (productos, eventos, marcas)

    FORMATO DE RESPUESTA:
    [ENTIDAD] → [CATEGORÍA] → [BREVE EXPLICACIÓN]

    TEXTO A ANALIZAR:
    {texto}
    """

    try:
        respuesta = cliente_gemini.models.generate_content(
            model="gemini-2.0-flash",
            contents=[prompt]
        )
        return respuesta.text
    except Exception as e:
        return f"❌ Error: {e}"

# Probar Gemini si está disponible
if cliente_gemini:
    print("🔍 Analizando con Gemini...")
    resultado_gemini = analizar_entidades_gemini(texto_ejemplo)
    print("\nAnálisis de Gemini:")
    print(resultado_gemini)
else:
    print("⏭️  Saltando análisis con Gemini (API Key no disponible)")

🔍 Analizando con Gemini...

Análisis de Gemini:
[María González] → [PERSONA] → [Nombre de persona]
[Universidad de Buenos Aires] → [ORGANIZACIÓN] → [Universidad]
[San Telmo] → [LUGAR] → [Barrio]
[MercadoLibre] → [ORGANIZACIÓN] → [Empresa]
[Obelisco] → [LUGAR] → [Monumento/Lugar específico]
[Carlos Pérez] → [PERSONA] → [Nombre de persona]
[Google Argentina] → [ORGANIZACIÓN] → [Empresa]
[Constitución] → [LUGAR] → [Estación de subte]
[La Boca] → [LUGAR] → [Barrio]



---
# PARTE 3: Interfaces Interactivas con Gradio

Crearemos interfaces web interactivas para probar nuestros modelos.

In [33]:
import gradio as gr

def interfaz_ner_transformers(texto):
    """Interfaz para el modelo de Transformers"""
    if not texto.strip():
        return {"text": "Ingresa un texto para analizar", "entities": []}

    if not ner_pipeline:
        return {"text": "Modelo no disponible", "entities": []}

    # Procesar con Transformers
    entidades = ner_pipeline(texto)

    # Formatear para Gradio HighlightedText
    entidades_gradio = []
    for ent in entidades:
        entidades_gradio.append({
            "entity": ent["entity_group"],
            "word": ent["word"],
            "start": ent["start"],
            "end": ent["end"],
            "score": ent["score"]
        })

    return {"text": texto, "entities": entidades_gradio}

# Ejemplos para la interfaz
ejemplos_arg = [
    "Me llamo Juan Pérez y trabajo en el Banco Nación en Buenos Aires.",
    "Cristina Kirchner fue presidenta de Argentina y vive en Santa Cruz.",
    "River Plate jugará contra Boca Juniors en el estadio Monumental.",
    "Lionel Messi nació en Rosario y jugó en el Barcelona.",
    "La Universidad de La Plata es muy prestigiosa en Argentina."
]

# Crear interfaz
demo_transformers = gr.Interface(
    fn=interfaz_ner_transformers,
    inputs=[
        gr.Textbox(
            label="📝 Texto a analizar",
            placeholder="Escribe aquí tu texto en español...",
            lines=4
        )
    ],
    outputs=[
        gr.HighlightedText(
            label="🎯 Entidades Identificadas",
            show_legend=True
        )
    ],
    title="NER con Transformers - Español (de argentina)",
    description="""
    **Modelo:** `mrm8488/bert-spanish-cased-finetuned-ner`

    Identifica entidades nombradas en textos en español:
    - 🧑 **PER**: Personas
    - 🌍 **LOC**: Lugares
    - 🏢 **ORG**: Organizaciones
    - 📦 **MISC**: Misceláneo
    """,
    examples=ejemplos_arg,
    allow_flagging="never",
    theme=gr.themes.Soft()
)

print("✅ Interfaz de Transformers creada")

✅ Interfaz de Transformers creada


In [34]:
# Interfaz para Gemini (solo si está disponible)
if cliente_gemini:
    def interfaz_ner_gemini(texto):
        """Interfaz para Gemini API"""
        if not texto.strip():
            return "Ingresa un texto para analizar"
        return analizar_entidades_gemini(texto)

    demo_gemini = gr.Interface(
        fn=interfaz_ner_gemini,
        inputs=[
            gr.Textbox(
                label="📝 Texto a analizar",
                placeholder="Escribe aquí tu texto en español...",
                lines=4
            )
        ],
        outputs=[
            gr.Textbox(
                label="🧠 Análisis de Gemini",
                lines=10
            )
        ],
        title="NER con Gemini - Análisis Detallado",
        description="""
        **Modelo:** Google Gemini 2.0 Flash

        Análisis avanzado de entidades nombradas con explicaciones contextuales
        optimizado para español argentino.
        """,
        examples=ejemplos_arg,
        allow_flagging="never",
        theme=gr.themes.Soft()
    )
    print("✅ Interfaz de Gemini creada")
else:
    print("⏭️  Interfaz de Gemini no creada (API Key no disponible)")

✅ Interfaz de Gemini creada


In [35]:
# Interfaz comparativa (solo si ambos están disponibles)
if ner_pipeline and cliente_gemini:
    def comparar_modelos(texto):
        """Compara resultados de ambos modelos"""
        if not texto.strip():
            return "Ingresa texto para comparar", "Ingresa texto para comparar"

        # Resultado Transformers
        entidades_tf = analizar_entidades_transformers(texto)
        resultado_tf = "TRANSFORMERS:\n\n"
        for ent in entidades_tf:
            resultado_tf += f"• {ent['texto']} → {ent['etiqueta']} (confianza: {ent['confianza']})\n"

        # Resultado Gemini
        resultado_gemini = "GEMINI:\n\n" + analizar_entidades_gemini(texto)

        return resultado_tf, resultado_gemini

    demo_comparativo = gr.Interface(
        fn=comparar_modelos,
        inputs=[
            gr.Textbox(
                label="📝 Texto a comparar",
                placeholder="Ingresa texto para ver la comparación...",
                lines=3
            )
        ],
        outputs=[
            gr.Textbox(label="Transformers", lines=8),
            gr.Textbox(label="Gemini", lines=8)
        ],
        title="⚔️ Comparación: Transformers vs Gemini",
        description="Compara los resultados de ambos enfoques lado a lado.",
        examples=[
            "Diego Maradona jugó en Boca Juniors y en el Napoli de Italia.",
            "El gobierno argentino anunció medidas desde Casa Rosada."
        ],
        allow_flagging="never"
    )
    print("✅ Interfaz comparativa creada")
else:
    print("⏭️  Interfaz comparativa no creada (requiere ambos modelos)")

✅ Interfaz comparativa creada


## 🚀 Lanzar Interfaces

Ejecuta las celdas siguientes para lanzar las interfaces interactivas:

In [36]:
# Lanzar interfaz de Transformers
if ner_pipeline:
    print("🚀 Lanzando interfaz de Transformers...")
    demo_transformers.launch(share=True, height=600)
else:
    print("❌ No se puede lanzar: modelo de Transformers no disponible")

🚀 Lanzando interfaz de Transformers...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6beca3f4ee9e77e94b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [37]:
# Lanzar interfaz de Gemini
if cliente_gemini:
    print("🚀 Lanzando interfaz de Gemini...")
    demo_gemini.launch(share=True, height=600)
else:
    print("❌ No se puede lanzar: API de Gemini no disponible")

🚀 Lanzando interfaz de Gemini...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7641b15a4be2d1bdc6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [38]:
# Lanzar interfaz comparativa
if ner_pipeline and cliente_gemini:
    print("🚀 Lanzando interfaz comparativa...")
    demo_comparativo.launch(share=True, height=600)
else:
    print("❌ No se puede lanzar: requiere ambos modelos disponibles")

🚀 Lanzando interfaz comparativa...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f095ac9f66fac0c59c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


---
# 🎓 EJERCICIOS

## 📝 Ejercicio 1: Personalización (BÁSICO)
1. Modifica los ejemplos para incluir más contexto argentino específico
2. Agrega 3 ejemplos nuevos con nombres de barrios porteños
3. Probá con texto de diferentes regiones de Argentina

## 🔧 Ejercicio 2: Análisis Comparativo (INTERMEDIO)
1. Crea una función que cuente cuántas entidades encuentra cada modelo
2. Implementa un sistema de métricas de tiempo de procesamiento
3. Analiza en qué casos cada modelo funciona mejor

## 🚀 Ejercicio 3: Extensiones Avanzadas (AVANZADO)
1. Implementa procesamiento en lote de múltiples textos
2. Crea una función de exportación de resultados a CSV
3. Desarrolla un sistema de filtrado por tipo de entidad

## 💡 Proyecto Integrador
Elegí una de estas aplicaciones y desarróllala:
- **Analizador de noticias argentinas**: Extrae personas y lugares de artículos
- **Procesador de CVs**: Identifica nombres, empresas y universidades
- **Análisis de redes sociales**: Detecta menciones de políticos y lugares

## 🤔 Preguntas de Reflexión
1. ¿Cuáles son las ventajas y desventajas de cada enfoque?
2. ¿En qué casos usarías un modelo local vs una API?
3. ¿Cómo evaluarías la precisión de los resultados?
4. ¿Qué consideraciones éticas debemos tener en cuenta?
5. ¿Cómo escalarías esta solución para procesar miles de documentos?

In [39]:
# 📝 ESPACIO PARA TUS EJERCICIOS
# Usa esta celda para experimentar y desarrollar tus soluciones

# Ejemplo: Función para contar entidades por tipo
def contar_entidades_por_tipo(texto):
    """Cuenta entidades por categoría usando Transformers"""
    if not ner_pipeline:
        return {}

    entidades = ner_pipeline(texto)
    conteo = {}

    for ent in entidades:
        tipo = ent['entity_group']
        if tipo in conteo:
            conteo[tipo] += 1
        else:
            conteo[tipo] = 1

    return conteo

# Probar la función
texto_prueba = "Juan Pérez trabaja en Google Argentina en Buenos Aires con María López."
print("📊 Conteo de entidades:")
print(contar_entidades_por_tipo(texto_prueba))


# 🧩 EJERCICIO 1 – Personalización de ejemplos argentinos

# 1) Redefinimos los ejemplos para la interfaz con más contexto argentino
ejemplos_arg = [
    # CABA y GBA
    "Me llamo Juan Pérez y trabajo como analista en el Banco Nación de Plaza de Mayo, en el microcentro porteño.",
    "Florencia González vive en Caballito y estudia en la Universidad de Buenos Aires, sede Ciudad Universitaria.",
    "Cristina Kirchner fue presidenta de la Argentina y suele pasar temporadas en El Calafate, en la provincia de Santa Cruz.",
    "River Plate jugará contra Boca Juniors en el estadio Monumental, en el barrio de Belgrano.",
    "Lionel Messi nació en Rosario, jugó muchos años en el Barcelona y actualmente vive en Miami.",

    # 2) + 3 ejemplos nuevos con barrios porteños
    "Martina López atiende su local de ropa en Palermo Soho y suele ir a reuniones en Puerto Madero.",
    "Carlos Díaz trabaja como médico en un hospital de Núñez y da clases en una clínica privada de Recoleta.",
    "La oficina central de la empresa tecnológica está en el barrio de Parque Patricios, cerca del Distrito Tecnológico de la Ciudad de Buenos Aires."
]

# 2) Textos de prueba por región de Argentina
textos_regionales = {
    "CABA – Línea de subte": """
    Tomás vive en Almagro y todos los días toma la línea B de subte
    desde Medrano hasta Leandro N. Alem para ir a su trabajo en el microcentro.
    """,
    "Provincia de Buenos Aires – Industria": """
    Julieta trabaja en una fábrica de autopartes en La Matanza
    y viaja seguido a La Plata para capacitarse en la Universidad Nacional de La Plata.
    """,
    "Interior – Turismo": """
    En sus vacaciones, Ana viajó a San Carlos de Bariloche,
    recorrió el Cerro Catedral y luego visitó Mendoza para hacer el Camino del Vino.
    """,
    "Norte argentino – Sector público": """
    Ricardo es docente en una escuela pública de Salta capital
    y participa en proyectos educativos junto con la Universidad Nacional de Tucumán.
    """
}

def probar_textos_regionales():
    """
    Usa los modelos ya definidos en el notebook:
    - analizar_entidades_transformers(texto)
    - analizar_entidades_gemini(texto)
    para comparar resultados en distintos contextos argentinos.
    """
    for nombre, texto in textos_regionales.items():
        print("=" * 80)
        print(f"Contexto: {nombre}")
        print("-" * 80)
        print("Texto:")
        print(texto.strip())
        print("\nResultado Transformers:")
        print(analizar_entidades_transformers(texto))

        # Gemini es opcional
        if 'cliente_gemini' in globals() and cliente_gemini:
            print("\nResultado Gemini:")
            resultado_gemini = analizar_entidades_gemini(texto)
            print(resultado_gemini)
        else:
            print("\n⏭️ Gemini no disponible (sin API Key configurada).")

# 🔄 RECONSTRUIR INTERFACES DE GRADIO CON LOS NUEVOS EJEMPLOS

import gradio as gr

# ======================
#  1) INTERFAZ – TRANSFORMERS
# ======================

def interfaz_transformers(texto):
    return analizar_entidades_transformers(texto)

demo_transformers = gr.Interface(
    fn=interfaz_transformers,
    inputs=gr.Textbox(lines=5, label="Texto de entrada"),
    outputs=gr.JSON(label="Entidades detectadas"),
    title="NER con Transformers (HuggingFace)",
    description="Modelo de NER basado en Transformers para textos en español.",
    examples=[[e] for e in ejemplos_arg]  # usa los nuevos ejemplos
)

# ======================
#  2) INTERFAZ – GEMINI
# ======================

def interfaz_gemini(texto):
    if 'cliente_gemini' not in globals() or not cliente_gemini:
        return {"error": "Gemini no está configurado (falta GOOGLE_API_KEY)"}
    return analizar_entidades_gemini(texto)

demo_gemini = gr.Interface(
    fn=interfaz_gemini,
    inputs=gr.Textbox(lines=5, label="Texto de entrada"),
    outputs=gr.Textbox(label="Salida (texto de Gemini)"),
    title="NER con Gemini",
    description="Interfaz de extracción de entidades usando Gemini.",
    examples=[[e] for e in ejemplos_arg]  # usa los nuevos ejemplos
)

# ======================
#  3) APP FINAL (TABs)
# ======================

app = gr.TabbedInterface(
    interface_list=[demo_transformers, demo_gemini],
    tab_names=["Transformers", "Gemini"]
)

app.launch()

📊 Conteo de entidades:
{'PER': 2, 'ORG': 1, 'LOC': 1}
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0d12e48525a5c7e644.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [40]:
# 🔧 EJERCICIO 2 – Análisis comparativo Transformers vs Gemini

import time
import re

def contar_entidades_transformers(texto: str) -> int:
    """
    Devuelve la cantidad de entidades detectadas por el modelo de Transformers.
    Usa la función analizar_entidades_transformers definida en el notebook.
    """
    entidades = analizar_entidades_transformers(texto)
    return len(entidades)

def contar_entidades_gemini_desde_salida(salida: str) -> int:
    """
    Estima la cantidad de entidades en la salida de Gemini.
    Asume el formato:
        [ENTIDAD] → [CATEGORÍA] → [EXPLICACIÓN]
    Cuenta líneas que contengan '→'.
    """
    if not isinstance(salida, str):
        return 0

    # Cada línea con al menos un "→" se considera una entidad
    lineas = [l.strip() for l in salida.splitlines() if "→" in l]
    return len(lineas)

def medir_modelo_transformers(texto: str):
    """Mide tiempo y cantidad de entidades de Transformers sobre un texto."""
    t0 = time.perf_counter()
    entidades = analizar_entidades_transformers(texto)
    t1 = time.perf_counter()
    return {
        "modelo": "Transformers",
        "cant_entidades": len(entidades),
        "tiempo_segundos": round(t1 - t0, 4),
        "entidades": entidades,
    }

def medir_modelo_gemini(texto: str):
    """Mide tiempo y cantidad de entidades (estimada) de Gemini sobre un texto."""
    if 'cliente_gemini' not in globals() or not cliente_gemini:
        return {
            "modelo": "Gemini",
            "error": "Cliente Gemini no disponible",
        }

    t0 = time.perf_counter()
    salida = analizar_entidades_gemini(texto)
    t1 = time.perf_counter()

    return {
        "modelo": "Gemini",
        "cant_entidades_estimada": contar_entidades_gemini_desde_salida(salida),
        "tiempo_segundos": round(t1 - t0, 4),
        "salida_cruda": salida,
    }

def comparar_modelos_en_texto(texto: str):
    """
    Ejecuta ambos modelos (si están disponibles) sobre el mismo texto
    y devuelve un diccionario con métricas comparativas.
    """
    resultado = {
        "texto": texto,
        "transformers": medir_modelo_transformers(texto)
    }

    if 'cliente_gemini' in globals() and cliente_gemini:
        resultado["gemini"] = medir_modelo_gemini(texto)
    else:
        resultado["gemini"] = {
            "modelo": "Gemini",
            "error": "Cliente Gemini no disponible",
        }

    return resultado

def comparar_modelos_en_varios_textos(lista_textos):
    """
    Aplica la comparación a una lista de textos y muestra un resumen amigable.
    """
    for i, texto in enumerate(lista_textos, start=1):
        print("=" * 100)
        print(f"TEXTO {i}")
        print("-" * 100)
        print(texto.strip())
        print("\nMÉTRICAS:")

        resultado = comparar_modelos_en_texto(texto)

        # Transformers
        t_res = resultado["transformers"]
        print(f"\n🔹 Transformers → entidades: {t_res['cant_entidades']}, "
              f"tiempo: {t_res['tiempo_segundos']} s")

        # Gemini
        g_res = resultado["gemini"]
        if "error" in g_res:
            print(f"🔸 Gemini → {g_res['error']}")
        else:
            print(f"🔸 Gemini → entidades (estimadas): {g_res['cant_entidades_estimada']}, "
                  f"tiempo: {g_res['tiempo_segundos']} s")

for nombre, texto in textos_regionales.items():
    print("======================================")
    print("Probando:", nombre)
    print(comparar_modelos_en_texto(texto))

Probando: CABA – Línea de subte
{'texto': '\n    Tomás vive en Almagro y todos los días toma la línea B de subte \n    desde Medrano hasta Leandro N. Alem para ir a su trabajo en el microcentro.\n    ', 'transformers': {'modelo': 'Transformers', 'cant_entidades': 5, 'tiempo_segundos': 0.0605, 'entidades': [{'texto': 'Tomás', 'etiqueta': 'PER', 'confianza': np.float32(0.999), 'posicion': (5, 10)}, {'texto': 'Almagro', 'etiqueta': 'LOC', 'confianza': np.float32(0.999), 'posicion': (19, 26)}, {'texto': 'Med', 'etiqueta': 'LOC', 'confianza': np.float32(0.999), 'posicion': (80, 83)}, {'texto': '##rano', 'etiqueta': 'LOC', 'confianza': np.float32(0.976), 'posicion': (83, 87)}, {'texto': 'Leandro N. Alem', 'etiqueta': 'LOC', 'confianza': np.float32(0.976), 'posicion': (94, 109)}]}, 'gemini': {'modelo': 'Gemini', 'cant_entidades_estimada': 6, 'tiempo_segundos': 1.2595, 'salida_cruda': '*   Tomás → PERSONA → Nombre de persona.\n*   Almagro → LUGAR → Barrio de Buenos Aires.\n*   línea B → MISCEL

In [41]:
# 🚀 EJERCICIO 3 – Procesamiento en lote, CSV y filtrado por tipo de entidad

import pandas as pd

def procesar_lote_transformers(lista_textos):
    """
    Procesa una lista de textos con el modelo de Transformers
    y devuelve un DataFrame con todas las entidades detectadas.

    Columnas:
    - id_texto
    - texto_resumen (primeros 200 caracteres)
    - entidad
    - tipo
    - confianza
    - inicio
    - fin
    """
    registros = []

    for idx, texto in enumerate(lista_textos, start=1):
        entidades = analizar_entidades_transformers(texto)
        texto_resumen = " ".join(texto.split())[:200]  # compacto y recorta

        for ent in entidades:
            registros.append({
                "id_texto": idx,
                "texto_resumen": texto_resumen,
                "entidad": ent["texto"],
                "tipo": ent["etiqueta"],
                "confianza": ent["confianza"],
                "inicio": ent["posicion"][0],
                "fin": ent["posicion"][1],
            })

    df = pd.DataFrame(registros)
    return df

def exportar_resultados_csv(df, nombre_archivo="resultados_ner_lote.csv"):
    """
    Exporta el DataFrame de resultados a un archivo CSV.
    """
    df.to_csv(nombre_archivo, index=False, encoding="utf-8")
    print(f"Archivo CSV guardado como: {nombre_archivo}")

def filtrar_por_tipo(df, tipos):
    """
    Filtra el DataFrame por uno o varios tipos de entidad.
    Ejemplos:
        filtrar_por_tipo(df, "PER")
        filtrar_por_tipo(df, ["PER", "ORG"])
    """
    if isinstance(tipos, str):
        tipos = [tipos]

    df_filtrado = df[df["tipo"].isin(tipos)].copy()
    return df_filtrado

# Ejemplo rápido de uso del ejercicio 3:
textos_ejemplo_lote = [
    "Juan Pérez trabaja en el Banco Nación en Buenos Aires.",
    "María Gómez vive en Córdoba y estudia en la Universidad Nacional de Córdoba.",
    "La planta industrial de Toyota está ubicada en Zárate, provincia de Buenos Aires."
]

df_resultados = procesar_lote_transformers(textos_ejemplo_lote)
print("👀 Vista previa de resultados en lote:")
display(df_resultados.head())

print("\n📎 Filtrando solo personas (PER):")
display(filtrar_por_tipo(df_resultados, "PER").head())

# Exportar a CSV (podés descargarlo desde el explorador de archivos del entorno)
exportar_resultados_csv(df_resultados, "ner_textos_lote.csv")



👀 Vista previa de resultados en lote:


,id_texto,texto_resumen,entidad,tipo,confianza,inicio,fin
0,1,Juan Pérez trabaja en el Banco Nación en Bueno...,Juan Pérez,PER,0.999,0,10
1,1,Juan Pérez trabaja en el Banco Nación en Bueno...,Banco Nación,ORG,1.000,25,37
2,1,Juan Pérez trabaja en el Banco Nación en Bueno...,Buenos Aires,LOC,0.999,41,53
3,2,María Gómez vive en Córdoba y estudia en la Un...,María Gómez,PER,1.000,0,11
4,2,María Gómez vive en Córdoba y estudia en la Un...,Córdoba,LOC,1.000,20,27



📎 Filtrando solo personas (PER):


,id_texto,texto_resumen,entidad,tipo,confianza,inicio,fin
0,1,Juan Pérez trabaja en el Banco Nación en Bueno...,Juan Pérez,PER,0.999,0,10
3,2,María Gómez vive en Córdoba y estudia en la Un...,María Gómez,PER,1.000,0,11


Archivo CSV guardado como: ner_textos_lote.csv


In [44]:
# PROYECTO INTEGRADOR – Procesador de CVs con NER + Gemini

import textwrap
import json

# 1) CV ficticio de ejemplo (basado en un perfil técnico electromecánico)
cv_ejemplo = """
Nombre: Patricio González
Dirección: Av Mitre 1, Buenos Aires (Avellaneda)
Teléfono: 1112345678
Correo electrónico: ejemplo@ejemplo.com

Perfil Profesional
Persona ordenada, responsable y con buena presencia, con experiencia en mantenimiento y
limpieza en el sector de la gastronomía y docencia en música. Actualmente cursando estudios
universitarios, busco oportunidades para aplicar mis habilidades y seguir desarrollándome en
un entorno laboral dinámico.

Experiencia Laboral
- Vinilo (Wilde):
  Empleado de Limpieza y Mantenimiento.
    Realización de tareas de limpieza general y mantenimiento de salón en un restaurante/boliche.
    Responsable de asegurar el orden y la higiene del local, contribuyendo a un ambiente
    agradable para los clientes.
    Responsable de cargar en el deposito la mercaderia ingresante al establecimiento.
- Profesor de Guitarra (Avellaneda)
    Clases personalizadas de guitarra para estudiantes principiantes e intermedios.
    Desarrollo de habilidades musicales y orientación en teoría básica y práctica instrumental.

Educación:
  Secundario Completo
  Universidad Nacional de Avellaneda
  Estudiante actual de [Guía universitario en turismo].

Habilidades:
  Organización y limpieza: experiencia en tareas de mantenimiento y orden.
  Responsabilidad: compromiso con la puntualidad y calidad en cada tarea.
  Habilidades interpersonales: experiencia en docencia y trato con estudiantes.
"""

# 2) Extraer entidades con Transformers

def extraer_entidades_cv(texto_cv: str):
    """
    Usa analizar_entidades_transformers para obtener las entidades del CV.
    Devuelve la lista de entidades tal como la formatea esa función.
    """
    return analizar_entidades_transformers(texto_cv)

# 3) Construir un perfil básico estructurado a partir de las entidades

def construir_perfil_basico_cv(entidades, texto_cv: str):
    """
    A partir de las entidades detectadas arma un perfil básico:
    - nombre (primer PER encontrado)
    - organizaciones (ORG únicas)
    - lugares (LOC únicos)
    - resumen corto (primeros N caracteres del CV)
    """
    perfil = {
        "nombre": None,
        "organizaciones": set(),
        "lugares": set(),
        "resumen_cv": " ".join(texto_cv.split())[:400]  # compactar y recortar
    }

    for ent in entidades:
        tipo = ent["etiqueta"]
        texto_ent = ent["texto"]

        if tipo == "PER" and perfil["nombre"] is None:
            perfil["nombre"] = texto_ent
        elif tipo == "ORG":
            perfil["organizaciones"].add(texto_ent)
        elif tipo == "LOC":
            perfil["lugares"].add(texto_ent)

    # Convertir sets a listas ordenadas para serializar
    perfil["organizaciones"] = sorted(perfil["organizaciones"])
    perfil["lugares"] = sorted(perfil["lugares"])

    return perfil

# 4) (Opcional) Resumen del CV usando Gemini

def resumir_cv_gemini(texto_cv: str):
    """
    Usa Gemini para generar un resumen del CV.
    Si Gemini no está disponible, devuelve un mensaje indicándolo.
    """
    if 'cliente_gemini' not in globals() or not cliente_gemini:
        return "Gemini no disponible. Configurá GOOGLE_API_KEY para usar esta función."

    prompt = f"""
    Sos un asistente de RRHH. Resumí el siguiente CV en un párrafo breve y claro
    en español, indicando:
    - Años aproximados de experiencia
    - Sector principal (industria, construcción, servicios, etc.)
    - Últimos puestos relevantes
    - Principales habilidades técnicas y blandas

    CV:
    {texto_cv}
    """

    try:
        respuesta = cliente_gemini.models.generate_content(
            model="gemini-2.0-flash",
            contents=[prompt]
        )
        return respuesta.text
    except Exception as e:
        return f"Error al llamar a Gemini: {e}"

# 5) Función integradora: procesa el CV completo

def procesar_cv(texto_cv: str):
    """
    Ejecuta el pipeline completo para un CV:
    - Extrae entidades con Transformers
    - Construye un perfil básico
    - Genera (opcionalmente) un resumen con Gemini
    """
    print("Extrayendo entidades con Transformers...")
    entidades = extraer_entidades_cv(texto_cv)

    print(f"Entidades detectadas: {len(entidades)}")

    print("\nConstruyendo perfil básico a partir de las entidades...")
    perfil = construir_perfil_basico_cv(entidades, texto_cv)

    print("\nPerfil básico estructurado:")
    print(json.dumps(perfil, indent=4, ensure_ascii=False))

    print("\nGenerando resumen del CV con Gemini (si está disponible)...")
    resumen = resumir_cv_gemini(texto_cv)
    print("\nResumen del CV:")
    print(resumen)

    return {
        "entidades": entidades,
        "perfil": perfil,
        "resumen": resumen
    }

# 6) Ejecutar el procesador sobre el CV de ejemplo

resultado_cv = procesar_cv(cv_ejemplo)


Extrayendo entidades con Transformers...
Entidades detectadas: 21

Construyendo perfil básico a partir de las entidades...

Perfil básico estructurado:
{
    "nombre": "Patricio González",
    "organizaciones": [
        "Universidad Nacional de Avellaneda"
    ],
    "lugares": [
        "##e",
        "##eda",
        "##ild",
        "##llan",
        "##llaneda",
        "Av Mitre 1",
        "Ave",
        "Buenos Aires",
        "W"
    ],
    "resumen_cv": "Nombre: Patricio González Dirección: Av Mitre 1, Buenos Aires (Avellaneda) Teléfono: 1112345678 Correo electrónico: ejemplo@ejemplo.com Perfil Profesional Persona ordenada, responsable y con buena presencia, con experiencia en mantenimiento y limpieza en el sector de la gastronomía y docencia en música. Actualmente cursando estudios universitarios, busco oportunidades para aplicar mis habilidades "
}

Generando resumen del CV con Gemini (si está disponible)...

Resumen del CV:
Patricio González posee aproximadamente 2 años de

---
# 🎯 Conclusión

¡Felicitaciones! Completaste el ejercicio de Reconocimiento de Entidades Nombradas.

## 📚 Lo que aprendiste:
- ✅ Implementar NER con modelos pre-entrenados
- ✅ Usar APIs de IA generativa para tareas de PLN
- ✅ Crear interfaces interactivas con Gradio
- ✅ Comparar diferentes enfoques de NER

## 🔄 Próximos pasos:
1. Experimentá con otros modelos de Hugging Face
2. Probá con textos de diferentes dominios
3. Implementa tu proyecto integrador
4. Compartí tus resultados con la clase

## 📖 Recursos adicionales:
- [Hugging Face Models](https://huggingface.co/models?pipeline_tag=token-classification&language=es)
- [Gradio Documentation](https://gradio.app/docs/)
- [Google AI Studio](https://ai.google.dev/)

---
**¡Éxito en tu trabajo integrador!** 🎓🚀